# Extracting Respiration Signals with Face Transformer

This notebook creates predictions of respiratory signals for all models trained with the face transformer architecture.

In [ ]:
import respiration.utils as utils

model_ids = [
    # '20240710_142159',
    # '20240710_194632',
    # '20240710_220756',
    # '20240711_194917',
    # '20240712_113946',
    # '20240712_163619',
    # '20240713_090928',
    '20240728_114332',
    '20240728_172805',
    '20240729_195756',
    '20240803_105616',
    '20240821_115511',
]

In [ ]:
device = utils.get_torch_device()

In [ ]:
import torch
from vit_pytorch import SimpleViT


def load_model(model_id: str) -> (SimpleViT, dict):
    model_dir = utils.dir_path('models', 'transformer', model_id)
    manifest_path = utils.join_paths(model_dir, 'manifest.json')
    manifest = utils.read_json(manifest_path)

    model = SimpleViT(
        image_size=manifest['image_size'],
        patch_size=manifest['image_patch_size'],
        num_classes=1,
        dim=manifest['embedding_dim'],
        heads=manifest['heads'],
        mlp_dim=manifest['mlp_dim'],
        depth=manifest['depth'],
    ).to(device)

    # Load the best model from the training process
    model_path = utils.join_paths(model_dir, manifest['trained_models'][-1]['model'])
    model.load_state_dict(torch.load(model_path, map_location=device))

    model.eval()

    return model, manifest

In [ ]:
from tqdm.auto import tqdm
from respiration.training import ScenarioLoader

predictions = []

for model_id in tqdm(model_ids):
    model, manifest = load_model(model_id)
    scenarios = manifest['testing_scenarios']

    image_dim = (256, 256)

    for inx, (subject, setting) in enumerate(scenarios):
        print(f'Processing {subject} - {setting}')
        num_frames = manifest['num_frames'] if 'num_frames' in manifest else 300
        loader = ScenarioLoader(subject, setting, num_frames, device)

        prediction = []

        for (frames, gt_classes) in loader:
            frames = utils.preprocess_frames(frames, image_dim, device).squeeze()
            # Disable gradient computation and reduce memory consumption.
            with torch.no_grad():
                outputs = model(frames).squeeze()
            prediction.extend(outputs.tolist())

        predictions.append({
            'subject': subject,
            'setting': setting,
            'model': model_id,
            'signal': prediction,
        })

In [ ]:
import pandas as pd

df = pd.DataFrame(predictions)

output_dir = utils.dir_path('outputs', 'signals', mkdir=True)

# Save the evaluation dataframe
csv_path = utils.join_paths(output_dir, 'transformer_predictions.csv')
df.to_csv(csv_path, index=False)

df.head()